In [1]:
from __future__  import print_function, division
import pylab as pl
import pandas as pd
import numpy as np
import os

%pylab inline
os.environ["PUI2018"] = "%s/PUI2018"%os.getenv("HOME")
if os.getenv ('PUI2018') is None:
    print ("Must set env variable PUI2018")
if os.getenv ('PUIDATA') is None:
    print ("Must set env variable PUI2018")

import os
import json

Populating the interactive namespace from numpy and matplotlib


In [2]:
def getCitiBikeCSV(datestring):
    print ("Downloading", datestring)
    ### First I will check that it is not already there
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        if os.path.isfile(datestring + "-citibike-tripdata.csv"):
            # if in the current dir just move it
            if os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA")):
                print ("Error moving file!, Please check!")
        #otherwise start looking for the zip file
        else:
            if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.zip"):
                if not os.path.isfile(datestring + "-citibike-tripdata.zip"):
                    os.system("curl -O https://s3.amazonaws.com/tripdata/" + datestring + "-citibike-tripdata.zip")
                ###  To move it I use the os.system() functions to run bash commands with arguments
                os.system("mv " + datestring + "-citibike-tripdata.zip " + os.getenv("PUIDATA"))
            ### unzip the csv 
            os.system("unzip " + os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv.zip")
            ## NOTE: old csv citibike data had a different name structure. 
            if '2014' in datestring:
                os.system("mv " + datestring[:4] + '-' +  datestring[4:] + 
                          "\ -\ Citi\ Bike\ trip\ data.csv " + datestring + "-citibike-tripdata.csv")
            os.system("mv " + datestring + "-citibike-tripdata.csv " + os.getenv("PUIDATA"))
    ### One final check:
    if not os.path.isfile(os.getenv("PUIDATA") + "/" + datestring + "-citibike-tripdata.csv"):
        print ("WARNING!!! something is wrong: the file is not there!")

    else:
        print ("file in place, you can continue")

In [ ]:
datestring = '201601'
getCitiBikeCSV(datestring)

file in place, you can continue


In [ ]:
datestring = '201601'
df = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')
# df is the dataframe where the content of the csv file is stored
df['date'] = pd.to_datetime(df['starttime'])
# note that with dataframes I can refer to variables as dictionary keys, 
# i.e. df['starttime'] or as attributes: df.starttime. 


In [ ]:
df.head()

# Idea:

### People are not very keen to follow up on their new years resolutions 

# Null Hypothesis

###  $H_{0}$ : Number of people riding bikes after January is same or more than that of January.

###  $H_{a}$ : Number of people riding bikes after January is less than that of January.


# Mathematical formulation of null hypothesis

###  $H_{0}$ : $Number of people_{riding bikes after January}$ >= $Number of people_{riding bikes in January}$

###  $H_{a}$ : $Number of people_{riding bikes after January}$ <  $Number of people_{riding bikes inJanuary}$

## significance level  $\alpha=0.05$


#### Here we will display plots by doing  a comparative analysis with the month of February

In [ ]:
df_rendered = df[['tripduration','date']]
df_rendered['date'] = df_rendered['date'].dt.week

In [ ]:
df_rendered.tail()

### Visualizing the weekwise ridership in the month of January

In [ ]:
fig = pl.figure(figsize(8,8))

#instad of plotting with matplotlib i.e. plot() i use the plot method in pandas
norm_w = 1
ax = ((df_rendered['date'].groupby([df_rendered['date']]).count()) / norm_w).plot(kind="bar",color='b')
ax.set_xlabel('Week number for the month of January')
ax.set_ylabel('Number of trips ')
ax.set_title('Bar chart showing number of trips taken weekwise in the month of January')


# We do a weekwise comparision of ridership in the month of February

### Downloading data for the month of February

In [ ]:

datestring = '201602'
getCitiBikeCSV(datestring)

In [ ]:
datestring = '201602'
df_feb = pd.read_csv(os.getenv("PUIDATA") + "/" + datestring + '-citibike-tripdata.csv')


In [ ]:
df_feb.head()

In [ ]:
df_feb['tripdate'] = pd.to_datetime(df_feb['starttime'], format='%m/%d/%Y %H:%M:%S')


In [ ]:
df_feb_reduced = df_feb[['tripdate','tripduration']]
df_feb_reduced['week'] = df_feb_reduced['tripdate'].dt.week
df_feb_reduced['week'] = df_feb_reduced['week']
df_feb_reduced.head()

In [ ]:
fig = pl.figure(figsize(8,8))

#instad of plotting with matplotlib i.e. plot() i use the plot method in pandas
norm_w = 1
ax = ((df_rendered['date'].groupby([df_rendered['date']]).count()) / norm_w).plot(kind="bar",color='b')
((df_feb_reduced['tripdate'].groupby([df_feb_reduced['tripdate'].dt.week]).count()) / norm_w).plot(kind="bar",color='g',ax=ax)
ax.set_xlabel('Week number for the month of January and February')
ax.set_ylabel('Number of weekwise trips taken in the month of January and February ')
ax.set_title('Bar chart showing number of trips taken weekwise in the month of January and February')
tmp = ax.xaxis.set_ticklabels(['1', '2', '3', '4', '5'], fontsize=20)
ax.legend(['January','February'])


## Interpretation
The number of trips taken in January appear to be **higher** than the number of trips taken in February overall

In [ ]:
fig = pl.figure(figsize(8,8))

#instad of plotting with matplotlib i.e. plot() i use the plot method in pandas
norm_w = 1

ax = ((df_feb_reduced['tripduration'].groupby([df_feb_reduced['tripdate'].dt.week]).sum()) / norm_w).plot(kind="bar",color='g')
((df_rendered['tripduration'].groupby([df_rendered['date']]).sum()) / norm_w).plot(kind="bar",color='b',ax=ax)
ax.set_xlabel('Week number for the months of January and February')
ax.set_ylabel('Tota tripduration for the months of January and February in seconds')

tmp = ax.xaxis.set_ticklabels(['1', '2', '3', '4', '5'], fontsize=20)
ax.legend(['February','January'])

## Interpretation
The sum of tripduration of all trips taken in January in seconds appears to be **lower** than the sum of tripduration of all trips taken in February overall

### Conclusion

An initial analysis by plots suggests that **more** trips were made in the month of **January** but **more** minutes were spent cycling in the month of **February**